Import necessary libraries

In [2]:
import pandas as pd
import re
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay, accuracy_score, precision_score, recall_score, f1_score

In [4]:
import openai
Preste_key = "Your key"
openai.api_key = Preste_key

Upload test dataset.We explained how to generate in dataset_generation file, here you can find a link to this file https://github.com/preste-ai/rnd-nlp-cot-chatgptdatagen/blob/main/dataset_generation.ipynb

In [5]:
test_set = pd.read_csv('Your test set', index_col=0)

Generate request for intent recognition

In [11]:
def generate_request(phrase):

    request = f"phrase: {phrase}.\n" \
    "définis l'intention de cette phrase. Les intentions possibles sont: 'réservation_de_restaurant', 'bagages_perdus', 'annuler_la_réservation', 'suggestion_de_voyage', 'visa_international', 'transfert', 'réserver_un_vol', 'location_de_voiture', 'réserver_un_hôtel', 'état_du_vol " \
                'output as a dictionary. No additional text. Use template\n' \
                '{"phrase":phrase, "label":intention}'
    return request

Run chat GPT on our data

In [1]:
def run_chatgpt(test_set):
    lst_of_sent = test_set['text-fr'].tolist()
    all_info_list = []
    for lst in lst_of_sent:
        tmp_req = generate_request(lst)
            # print(tmp_req)
        completion = openai.ChatCompletion.create(
              model="gpt-3.5-turbo",
              messages=[
                {"role": "user", "content": tmp_req}
              ]
            )
        print(completion.choices[0].message.content)

        tmp_res = completion.choices[0].message.content

        all_info_list.append(tmp_res)
        return all_info_list

In [ ]:
all_info_list = run_chatgpt(test_set)

Create a column with predictions

In [27]:
def create_table(all_info_list):
    lst_phrases = []
    lst_labels = []
    for i, dct in enumerate(all_info_list):
        try:
            dct = eval(dct)
        except (SyntaxError) as error:
            print(error)
            print(i, dct)
            continue
        lst_phrases.append(dct["phrase"])
        lst_labels.append(dct["label"])
    gpt_data = {"Phrases": lst_phrases, "Labels": lst_labels}
    gpt_df = pd.DataFrame(gpt_data)
    gpt_df["Labels_encoding"] = gpt_df['Labels'].replace({'réservation_de_restaurant' : 0, 
               'bagages_perdus' : 1, 
               'annuler_la_réservation' : 2, 
               'suggestion_de_voyage' : 3, 
               'visa_international' : 4, 
               'transfert' : 6, 
               'réserver_un_vol' : 7, 
               'location_de_voiture' : 8, 
               'réserver_un_hôtel' : 9, 
               'état_du_vol' : 5})
    return gpt_df

In [28]:
gpt_df = create_table(all_info_list)

Get metrics

In [ ]:
def get_metrics(y_pred, y_test): 
    test_report = classification_report(y_test, y_pred, output_dict=True)

    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')

    return test_report, accuracy, precision, recall, f1

In [ ]:
test_report, accuracy, precision, recall, f1 = get_metrics(test_set["labels"], gpt_df["Labels_encoding"])

In [ ]:
print("Accuracy: ", accuracy)
print("Precision: ", precision)
print("Recall: ", recall)
print("F1: ", f1)